# Optimal Location for an Italian Restaurant in Manhattan

## Introduction

In this project, I will try to find an optimal location(postal code) for an Italian restaurant in Manhattan, NY.

I will try to find the location which is generally growded and where there are many restaurants but not a lot of Italian restaurants. Therefore, the first condition will be where people have a good access to many different restaurants, and the second condition will be where there are as few Italian restaurants as possible.

I will use methods that I learned through IBM Data Science courses to explore and utilize the dataset from Foursquare.

## Data

I will need the following information to solve the problem stated above.

* the number of any restaurants in Manhattan, NY and their exact location.  
* the number of Italian restaurants in Manhattan, NY and their exact location.

Following data sources will be needed to extract/generate the required information:

* Dataset related to restaurants will be extracted from FourSquare API.
    * name, categories of the restaurants, latitude and longitude of the restraurants.
* Dataset related to geographical coordinates of neighborhood will be generated using the data provided by IBM.
    * postal code, latitude, longitude of each neighborhood.

#### Sample Datasets

Example of the dataset of venues(including restaurants) in Manhattan.  
<img src='data_example_2.JPG'>

The dataset above has names of the venues, categories of them and geographical coordinates of each venue. The columns are name, categories, lat and lng.

Example of the dataset of geographical coordinates of neighborhoods in Manhattan.  
<img src='data_example.JPG'>

The dataset above has names of neighborhoods and geographical coordinates of the neighborhoods. The columns are Borough, Neighborhood, Latitude and Longitude. The dataset has 40 different neighborhoods in Manhattan.

## Data Exploration

I will use the datset from IBM which has the names of neighborhoods in Manhattan.

#### 1. Importing libraries

In [1]:
# importing necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### 2. Importing and exploring the data

In [2]:
import requests

newyork_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
newyork_data = requests.get(newyork_url).json()

In [3]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

The dataset above has all of the boroughs in New York, so I will break it down to Manhattan.

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
manhattan_data.shape

(40, 4)

In [7]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [8]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<img src='manhattan_neighborhoods.JPG'>

I will extract data of venues in Manhattan from FourSquare API.

In [13]:
CLIENT_ID = 'S30REPLDJD5AFTGWKGT1B3KI504XUDFACMAV2A53PS44WBIB' # Foursquare ID
CLIENT_SECRET = 'NEUKBATTRMSTEPQ05DS1JDLXMTP1KOM5S4BNR02JJLREVWMT' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S30REPLDJD5AFTGWKGT1B3KI504XUDFACMAV2A53PS44WBIB
CLIENT_SECRET:NEUKBATTRMSTEPQ05DS1JDLXMTP1KOM5S4BNR02JJLREVWMT


I will see what kind of venues are in the first neighborhood in Manhattan, which is Marble Hill.

In [14]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [15]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.  
First, let's create the GET request URL. Name your URL url.

In [17]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=S30REPLDJD5AFTGWKGT1B3KI504XUDFACMAV2A53PS44WBIB&client_secret=NEUKBATTRMSTEPQ05DS1JDLXMTP1KOM5S4BNR02JJLREVWMT&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [24]:
results = requests.get(url).json()

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-26-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Arturo's,Pizza Place,40.874412,-73.910271
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Rite Aid,Pharmacy,40.875467,-73.908906
4,Subway,Sandwich Place,40.874667,-73.909586


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


The results above are all in Marble Hill. They are examples of the dataset I will use. Let's start extracting venues in all neighborhoods this time.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [30]:
print(manhattan_venues.shape)
manhattan_venues.head()

(2989, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
4,Marble Hill,40.876551,-73.91066,Subway,40.874667,-73.909586,Sandwich Place


In [36]:
manhattan_italian = manhattan_venues[manhattan_venues['Venue Category'] == 'Italian Restaurant']
manhattan_italian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
134,Washington Heights,40.851903,-73.936900,Saggio Restaurant,40.851423,-73.939761,Italian Restaurant
214,Inwood,40.867684,-73.921210,Trattoria Inwood,40.868416,-73.918647,Italian Restaurant
263,Hamilton Heights,40.823604,-73.949688,Fumo,40.821412,-73.950499,Italian Restaurant
322,Manhattanville,40.816934,-73.957385,Pisticci Ristorante,40.814015,-73.960266,Italian Restaurant
331,Manhattanville,40.816934,-73.957385,Bettolona,40.814084,-73.959574,Italian Restaurant


In [44]:
# create map of italian restaurants in manhattan using latitude and longitude values
map_italian = folium.Map(location=[latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(manhattan_italian['Venue Latitude'], manhattan_italian['Venue Longitude'], manhattan_italian['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_italian)  
    
map_italian

<img src='italian_map.JPG'>

In [47]:
manhattan_italian.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Carnegie Hill,1,1,1,1,1,1
Chelsea,3,3,3,3,3,3
Civic Center,2,2,2,2,2,2
Clinton,5,5,5,5,5,5
East Village,2,2,2,2,2,2
Financial District,2,2,2,2,2,2
Flatiron,7,7,7,7,7,7
Gramercy,3,3,3,3,3,3
Greenwich Village,12,12,12,12,12,12


According to the dataframe above, Greenwich Village, West Village and Upper East Side, Yorkville are the three neighborhoods that have the most Italian restaurants.

I will also take a look at the Manhattan dataset to see which neighborhoods are crowded and which are not. This will help find out which areas can expect many customers.

In [48]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,61,61,61,61,61,61
Carnegie Hill,71,71,71,71,71,71
Central Harlem,37,37,37,37,37,37
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,85,85,85,85,85,85
Clinton,100,100,100,100,100,100
East Harlem,39,39,39,39,39,39
East Village,100,100,100,100,100,100


Comparing the two datasets, it's possible to see which areas have how many venues and how many italian restaurants. According to the dataset of Manhattan data, Chelsea, Chinatown, Clinton, East Village, Financial District, Flatiron, Greenwich Village, Lenox Hill, Little Italy, Midtown, Midtown South, Noho, Soho, West Village have the most venues. Among the states neighborhoods, Greenwich Village and West Village already have many Italian restaurants.

If the neighborhoods are not stated in italian restaurant dataset, it means it doesn't have any Italian restaurants in the neighborhoods. This can lead people to think where there are many venues but no Italian restaurants could be the optimal locations to open a new Italian restaurants.

In [50]:
group_italian = manhattan_italian.groupby('Neighborhood').count()
group_manhattan = manhattan_venues.groupby('Neighborhood').count()

In [51]:
group_italian.index

Index(['Carnegie Hill', 'Chelsea', 'Civic Center', 'Clinton', 'East Village',
       'Financial District', 'Flatiron', 'Gramercy', 'Greenwich Village',
       'Hamilton Heights', 'Hudson Yards', 'Inwood', 'Lenox Hill',
       'Lincoln Square', 'Little Italy', 'Manhattan Valley', 'Manhattanville',
       'Midtown', 'Midtown South', 'Morningside Heights', 'Murray Hill',
       'Noho', 'Soho', 'Sutton Place', 'Tribeca', 'Tudor City', 'Turtle Bay',
       'Upper East Side', 'Upper West Side', 'Washington Heights',
       'West Village', 'Yorkville'],
      dtype='object', name='Neighborhood')

In [52]:
group_manhattan.index

Index(['Battery Park City', 'Carnegie Hill', 'Central Harlem', 'Chelsea',
       'Chinatown', 'Civic Center', 'Clinton', 'East Harlem', 'East Village',
       'Financial District', 'Flatiron', 'Gramercy', 'Greenwich Village',
       'Hamilton Heights', 'Hudson Yards', 'Inwood', 'Lenox Hill',
       'Lincoln Square', 'Little Italy', 'Lower East Side', 'Manhattan Valley',
       'Manhattanville', 'Marble Hill', 'Midtown', 'Midtown South',
       'Morningside Heights', 'Murray Hill', 'Noho', 'Roosevelt Island',
       'Soho', 'Stuyvesant Town', 'Sutton Place', 'Tribeca', 'Tudor City',
       'Turtle Bay', 'Upper East Side', 'Upper West Side',
       'Washington Heights', 'West Village', 'Yorkville'],
      dtype='object', name='Neighborhood')

In [71]:
lst_yes = [] # neighborhoods with italian restaurants
lst_no = [] # neighborhoods with no italian restaurants
for i in group_manhattan.index:
    if i in group_italian.index:
        lst_yes.append(i)
    else:
        lst_no.append(i)

In [90]:
index = group_italian[group_italian['Venue'] == 1].index
lst_1 = []
for i in index:
    lst_1.append(i)

lst_1

['Carnegie Hill',
 'Hamilton Heights',
 'Inwood',
 'Manhattan Valley',
 'Morningside Heights',
 'Tudor City',
 'Washington Heights']

In [75]:
pd.concat([group_manhattan.loc[lst_no], group_manhattan.loc[lst_1]])

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,61,61,61,61,61,61
Central Harlem,37,37,37,37,37,37
Chinatown,100,100,100,100,100,100
East Harlem,39,39,39,39,39,39
Lower East Side,37,37,37,37,37,37
Marble Hill,23,23,23,23,23,23
Roosevelt Island,26,26,26,26,26,26
Stuyvesant Town,16,16,16,16,16,16
Carnegie Hill,71,71,71,71,71,71


The dataset above shows the venues that have 0 or 1 italian restaurants.

In [65]:
group_manhattan.loc[lst_yes].loc[['Greenwich Village', 'West Village', 'Upper East Side', 'Yorkville', 'Lenox Hill']]

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Greenwich Village,100,100,100,100,100,100
West Village,100,100,100,100,100,100
Upper East Side,77,77,77,77,77,77
Yorkville,96,96,96,96,96,96
Lenox Hill,100,100,100,100,100,100


The dataset above shows the venues that have more than 8 Italian restaurants in the neighborhoods.

## Conclusion

In [85]:
final_1 = manhattan_data[manhattan_data['Neighborhood'].isin(['Greenwich Village', 'West Village', 'Upper East Side', 'Yorkville', 'Lenox Hill'])]
final_1

,Borough,Neighborhood,Latitude,Longitude
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118
10,Manhattan,Lenox Hill,40.768113,-73.958860
18,Manhattan,Greenwich Village,40.726933,-73.999914
24,Manhattan,West Village,40.734434,-74.006180


In [92]:
map_final_1 = folium.Map(location=[latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(final_1['Latitude'], final_1['Longitude'], final_1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final_1)  
    
map_final_1

<img src='final1.JPG'>  

The map above shows where there are crowded by random customers and also many Italian restaurants.

In [94]:
final_2 = manhattan_data[manhattan_data['Neighborhood'].isin(['Chinatown', 'Washington Heights', 'Carnegie Hill'])]
final_2

,Borough,Neighborhood,Latitude,Longitude
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
30,Manhattan,Carnegie Hill,40.782683,-73.953256


In [95]:
# create map of italian restaurants in manhattan using latitude and longitude values
map_final_2 = folium.Map(location=[latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(final_2['Latitude'], final_2['Longitude'], final_2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final_2)  
    
map_final_2

<img src='final2.JPG'>

The map above shows where there are somewhat crowded by random customers but 0 or 1 Italian restaurants.

For those who want to open a new Italian restuarants where there are many customers but no italian restaurants so they don't have to compete with other Italian restaurants, it could be a good idea to open it in the neighborhoods shown in the first dataset above: Chinatown, Washington Heights, or Carnegie Hill. However, if you look at the name of he nieghborhoods that don't have Italian restaurants, there is Chinatown. It is easily assumed that there will be Chinese people who are more familiar with Asian food rather than Italian food. That means, it might not be a good idea to open a new Italian restaurants because customers there could not pick Italian restaurants for their lunch or dinner. In conclusion, opening a new Italian restaurants in neighborhoods like Chinatown can be very challenging, but it can be very successful if they can appeal to customers like Chinese.

For thoese who want to open a new Italian restaurants where there are many Italian restaurants around, the neighborhoods stated in the second dataset can be the optimal locations: Greenwich Village, West Village, Lenox Hill, Yorkville or Upper East Side. In these locations, it might be very competitive because they already have many Italian restaurtns. However, that can also mean that people are expecting to easily see Italian restaurtns in the neighborhoods, so it can lead the restaurant owners to have customers who already know they want to have Italian restaurants. 

In conclusion, those who want to open a new Italian restaurant in Manhattan, they should decide where they want to open it according to the characteristics of the neighborhood: whether they want to open it where they don't have to compete with other Italian restaurants or they want to open it where they can easily have customers who want Itailian foods.